In [705]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img

In [706]:
IMAGE_SIZE = (260, 260, 3)
BATCH_SIZE = 16
EPOCHS = 50
LEARNING_RATE = 0.001

In [ ]:
# Load image
DATASET_CLEAN_PATH = os.path.join("Dataset", "Dataset Clean")

image_list = []
class_list = []
class_names = {}

for idx, label in enumerate(os.listdir(DATASET_CLEAN_PATH)):
    for filename in os.listdir(os.path.join(DATASET_CLEAN_PATH, label)):
        image_path = os.path.join(DATASET_CLEAN_PATH, label, filename)
        image = load_img(image_path, color_mode="rgb", target_size=IMAGE_SIZE[:2])
        image = np.asarray(image) / 255.
        image_list.append(image)
        class_list.append(idx)

    class_names[idx] = label
    print(f"Class {label} loaded successfully")

In [ ]:
from matplotlib import pyplot as plt
import random

plt.figure(figsize=(11, 7))
for i in range(9):
    plt.subplot(3, 3, i+1)
    randomNum = random.randrange(0, len(image_list))
    plt.imshow(image_list[randomNum])
    plt.axis('off')
    plt.title(f"{class_names[class_list[randomNum]]}: Image {randomNum}")

plt.show()

In [709]:
from tensorflow.keras.utils import to_categorical

image_list = np.array(image_list)
class_list = np.array(class_list)
class_list = to_categorical(class_list, num_classes=len(class_names))

In [ ]:
from sklearn.model_selection import train_test_split

# Train Test Split
X_train, X_temp, y_train, y_temp = train_test_split(image_list, class_list, test_size=0.3, random_state=42, shuffle=True, stratify=class_list)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True, stratify=y_temp)

print(f"Length of training data {len(y_train)}")
print(f"Length of testing data {len(y_test)}")
print(f"Length of validation data {len(y_val)}")

In [711]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2

base_model = EfficientNetB0(include_top=False, input_shape=IMAGE_SIZE, weights='imagenet')

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(len(class_names), activation='softmax')
])

In [712]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    horizontal_flip=True,
)

datagen.fit(X_train)
train_ds = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

In [ ]:
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=LEARNING_RATE)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=3)

history = model.fit(
    datagen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    callbacks=[early_stopping],
    steps_per_epoch=len(X_train) // BATCH_SIZE
)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Accuracy History")

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Loss History")

plt.legend(loc='lower right')
plt.show()